In [86]:
import pandas as pd
import numpy as np
from  sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor, BaggingRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import time
import pandas_profiling as pp
%matplotlib inline

In [78]:
def root_mean_square_error(model,predictedval,actualval):
    print(model + ' Value ',np.sqrt(mean_squared_error(predictedval, actualval)))
    print(model + ' r2_score ',r2_score(predictedval,actualval))
    return np.sqrt(mean_squared_error(predictedval, actualval))

In [79]:
train = pd.read_csv('PredictUpVoteCount/train_NIR5Yl1.csv')
train = train.drop(train[train.Views > 3000000].index)

In [80]:
labelencoder_X = LabelEncoder()
train['Tag'] = labelencoder_X.fit_transform(train['Tag'])
train.drop(['ID','Username'], axis=1,inplace =True)
target = train['Upvotes']

In [81]:
from sklearn.preprocessing import Binarizer
bn = Binarizer(threshold=7)
pd_watched = bn.transform([train['Answers']])[0]
train['pd_watched'] = pd_watched

In [42]:
pp.ProfileReport(train)

In [82]:
feature_names = [x for x in train.columns if x not in ['Upvotes']]

x_train, x_val, y_train, y_val = train_test_split(train[feature_names], target,test_size = 0.22,random_state =205)
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_val = sc_X.transform(x_val)

poly_reg = PolynomialFeatures(degree = 4,interaction_only=False, include_bias=True)
X_poly = poly_reg.fit_transform(x_train)
poly_reg.fit(x_train, y_train)
x_val1 = x_val.copy()
x_val = poly_reg.fit_transform(x_val)

lin_reg_1 = linear_model.LassoLars(alpha=0.021,max_iter=150)
#rf = RandomForestRegressor()
#gbr = GradientBoostingRegressor()
xgbr = xgb.XGBRegressor()

lin_reg_1.fit(X_poly, y_train)
#rf.fit(X_poly, y_train)
#gbr.fit(X_poly, y_train)
xgbr.fit(X_poly, y_train)

# predicitng 
pred_val = lin_reg_1.predict(x_val)
root_mean_square_error('Lasso',pred_val,y_val)

# pred_val2 = rf.predict(x_val)
# root_mean_square_error('RandomForest',pred_val2,y_val)

# pred_val3 = gbr.predict(x_val)
# root_mean_square_error('GradientBoosting',pred_val3,y_val)

pred_val4 = xgbr.predict(x_val)
root_mean_square_error('XGBoost',pred_val4,y_val)

C:\Users\msdaw\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\msdaw\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\msdaw\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  
C:\Users\msdaw\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


KeyboardInterrupt: 

In [ ]:
bagging_reg = BaggingRegressor(base_estimator=DecisionTreeRegressor(), max_samples=1.0, bootstrap=False,n_estimators=50,random_state=0,n_jobs=-1)
t0 = time.time()
bagging_reg.fit(x_train, y_train)
print('time taken:')
print(time.time()-t0)
pred_val5 = bagging_reg.predict(x_val)
root_mean_square_error('bagging_reg',pred_val5,y_val)

In [43]:
df_preds = pd.DataFrame({
    'LassoPreds':abs(pred_val),
    'XGBPreds':pred_val4,
    'ActualPreds':y_val
})

df_preds.tail(10)

,LassoPreds,XGBPreds,ActualPreds
44998,39.517042,32.971127,18.0
96093,35.302584,34.199360,4.0
134594,120.118572,124.526970,139.0
65584,94.599654,123.653915,103.0
300757,31.523389,88.706802,24.0
290437,52.520911,27.156178,16.0
125928,109.940491,127.109085,113.0
70946,7600.319956,7751.013184,6797.0
264417,71.810473,154.642014,80.0
247595,39.608878,34.199360,19.0


In [52]:
df_preds.tail(10)

,LassoPreds,XGBPreds,ActualPreds,minValue,meanValue,medianValue,RoundedminValue
44998,39.517042,32.971127,18.0,32.971127,36.244084,39.517042,33.0
96093,35.302584,34.199360,4.0,34.199360,34.750972,35.302584,34.0
134594,120.118572,124.526970,139.0,120.118572,122.322771,120.118572,120.0
65584,94.599654,123.653915,103.0,94.599654,109.126785,94.599654,95.0
300757,31.523389,88.706802,24.0,31.523389,60.115095,31.523389,32.0
290437,52.520911,27.156178,16.0,27.156178,39.838544,52.520911,27.0
125928,109.940491,127.109085,113.0,109.940491,118.524788,109.940491,110.0
70946,7600.319956,7751.013184,6797.0,7600.319956,7675.666570,7600.319956,7600.0
264417,71.810473,154.642014,80.0,71.810473,113.226243,71.810473,72.0
247595,39.608878,34.199360,19.0,34.199360,36.904119,39.608878,34.0


In [46]:
df_preds['minValue']=df_preds[['LassoPreds','XGBPreds']].min(axis=1)
root_mean_square_error('miValue',df_preds['minValue'],y_val)

miValue Value  1041.6476499060964
miValue r2_score  0.8726873134749099


1041.6476499060964

In [47]:
df_preds['meanValue']=df_preds[['LassoPreds','XGBPreds']].mean(axis=1)
root_mean_square_error('meanValue',df_preds['meanValue'],y_val)

meanValue Value  1132.6829237898455
meanValue r2_score  0.8715744408035823


1132.6829237898455

In [48]:
df_preds['medianValue']=df_preds[['LassoPreds']].median(axis=1)
root_mean_square_error('medianValue',df_preds['medianValue'],y_val)

medianValue Value  1030.5685625122564
medianValue r2_score  0.8920575387346987


1030.5685625122564

In [51]:
df_preds['RoundedminValue']=df_preds[['LassoPreds','XGBPreds']].min(axis=1).round(0)
root_mean_square_error('RoundedminValue',df_preds['RoundedminValue'],y_val)

RoundedminValue Value  1041.6474134418736
RoundedminValue r2_score  0.8726878406650616


1041.6474134418736

In [68]:
#stacked_val_preds = np.column_stack((pred_val,pred_val2,pred_val3,pred_val4))
#stacked_val_preds = np.column_stack((pred_val,pred_val4))
stacked_val_preds = df_preds['minValue'].values.reshape(-1,1)

xgbr_metamodel = xgb.XGBRegressor()
xgbr_metamodel.fit(stacked_val_preds,y_val)
pred_test = xgbr_metamodel.predict(stacked_val_preds)
root_mean_square_error('xgbr_metamodel',pred_test,y_val)

# linear_metamodel = linear_model.LinearRegression()
# linear_metamodel.fit(stacked_val_preds,y_val)
# pred_test = linear_metamodel.predict(stacked_val_preds)
# root_mean_square_error('LinearRegMetaModel',pred_test,y_val)

C:\Users\msdaw\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


xgbr_metamodel Value  586.0714197806734
xgbr_metamodel r2_score  0.9669438265026951
xgbr_metamodel_abs Value  586.0714197806734
xgbr_metamodel_abs r2_score  0.9669438265026951


586.0714197806734

In [70]:
df_stacked_preds = pd.DataFrame({
    'xgbr_metamodel':pred_test,
    'ActualPreds':y_val
})

df_stacked_preds.head(10)

,xgbr_metamodel,ActualPreds
132133,92.179848,58.0
280391,106.326164,95.0
60588,43.023830,22.0
73372,27.364485,21.0
177614,27.364485,10.0
113956,331.186340,356.0
128235,27.364485,10.0
139340,37.903820,19.0
21479,331.186340,440.0
40749,27.364485,7.0


## Test Submission

In [71]:
df_test = pd.read_csv('PredictUpVoteCount/test_8i3B3FC.csv')
ids = df_test['ID']
df_test.drop(['ID','Username'], axis=1,inplace =True)

labelencoder_X = LabelEncoder()
df_test['Tag'] = labelencoder_X.fit_transform(df_test['Tag'])

bn = Binarizer(threshold=7)
pd_watched = bn.transform([df_test['Answers']])[0]
df_test['pd_watched'] = pd_watched

df_test = sc_X.fit_transform(df_test)
df_test_poly = poly_reg.fit_transform(df_test)

y_pred1 = lin_reg_1.predict(df_test_poly)
y_pred4 = xgbr.predict(df_test_poly)

C:\Users\msdaw\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\msdaw\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [73]:
df_submitpreds = pd.DataFrame({
    'LassoPreds':abs(y_pred1),
    'XGBPreds':y_pred4
})

df_submitpreds.tail(10)

,LassoPreds,XGBPreds
141438,206.389195,199.417496
141439,33.470762,26.450941
141440,115.552068,168.637711
141441,30.961092,26.450941
141442,45.351340,75.249504
141443,46.412767,59.658009
141444,169.973314,254.317688
141445,62.994995,99.657135
141446,36.205510,34.199360
141447,20.245598,34.199360


In [74]:
df_submitpreds['minValue']=df_submitpreds[['LassoPreds','XGBPreds']].min(axis=1)

sub_stacked_val_preds = df_submitpreds['minValue'].values.reshape(-1,1)

sub_pred_test = xgbr_metamodel.predict(sub_stacked_val_preds)

df_submitpreds['FinalSubmission'] = sub_pred_test

In [75]:
df_submitpreds.head(10)

,LassoPreds,XGBPreds,minValue,FinalSubmission
0,206.698068,313.880188,206.698068,255.836441
1,120.647439,75.368927,75.368927,77.823189
2,49.809211,73.114166,49.809211,53.879349
3,38.855724,51.836262,38.855724,37.903820
4,278.421438,226.894028,226.894028,255.836441
5,31.735543,36.690605,31.735543,32.723946
6,12.288668,26.450941,12.288668,27.364485
7,79.272229,99.657135,79.272229,79.577126
8,69.060005,91.646957,69.060005,68.520561
9,28.923874,26.450941,26.450941,27.364485


In [76]:
submission = pd.DataFrame({'ID': ids,
                           'Upvotes':sub_pred_test
                           })

submission.to_csv("final_sub8.csv",index=False)